## Gather

依照課程提供，我有兩份資料可以直接用 `pandas` 打開
- twitter-archive-enhanced.csv
- image-predictions.tsv

另外還有一份是透過 twitter api 取得資訊，取用的程式我寫在 `tweet.py`，得到一份資料存成 txt 檔，再用 `pandas` 進行讀取。
- tweet_json.txt

## Access

現在我們手上有三份檔案，其中 Tweet JSON 的資料是用 Twitter archive enhanced table 的 tweet_id 做為條件，透過 twitter api 所取得的資訊，仔細觀察 Twitter archive enhanced table 和 Tweet json table，這兩張 table 其實有許多重複欄位；此外，由於資料來自 Twitter，因此欄位可參考 Twitter document：[Tweet Objects](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html)，接下來我們先用 document 來評估 Tweet JSON。

#### Tweet JSON table

**Quality issues**:

_missing data_

- contributors、coordinates、geo 完全沒有資料，因此我們不考慮這些欄位。
- extended_entities 紀錄附件資訊，缺少這個欄位的資料，我們可以判定該推文並沒有圖片，因此不考慮這些沒有圖片的數據。
- in_reply_to_screen_name、in_reply_to_status_id、in_reply_to_status_id_str、in_reply_to_user_id、in_reply_to_user_id_str 是同一個群組，當此訊息是回覆推文，會紀錄相關的訊息，那我們只考慮原始推文的資料，所以資料是回覆推文就要刪除。
- possibly_sensitive、possibly_sensitive_appealable 紀錄敏感性資料，我認為這些欄位跟這次的主題沒有太大關係，不用特地保留。
- place 顯示推文地點資訊，但是只有 1筆，因此不保留此欄位。
- quoted_status、quoted_status_id、quoted_status_id_str、quoted_status_permalink 是同一群組，當訊息是引用推文，這些欄位就會紀錄相關訊息，一樣我們不要這些數據。
- retweeted_status: Retweets can be distinguished from typical Tweets by the existence of a retweeted_status attribute. 因此只要是轉發的數據，我們也不要。

接下來看其它欄位：

- display_text_range 表示文字範圍，不用這個欄位
- entities 原始推文的相關資訊，由於有 image prediction 找到相關圖片預測，因此也不用這個欄位
- favorited 全部都是 False，基本上也不用這個欄位
- id_str 型別錯誤，應為 string object
- is_quote_status: Indicates whether this is a Quoted Tweet. 因此只要是 True 我們不要
- lang 型別錯誤，應為 category datatype
- source 夾雜 html 標籤
- retweeted 都是 False，不用這個欄位
- truncated 都是 False，不用這個欄位
- user 都是同一個帳號 @WeRateDogs，這個欄位也可以移除

#### Twitter archive enhanced table

由於 Twitter archive enhanced table 和 Tweet JSON table 有很多欄位資訊相同，這裡可以利用整理好的 Tweet JSON table 和 Twitter archive enhanced table 做個合併，同時只留下必要數據。

**Quaility issues**

_missing data_
- in_reply_to_status_id、in_reply_to_user_id 欄位資訊在 Tweet JSON 也有，我們可以合併完 table 後再來檢查數據是否為回覆訊息，是的話就要刪除
- retweeted_status_id、retweeted_status_user_id、retweeted_status_timestamp，推測是轉發推文id，可以等 archive table 和 tweet json 資料合併處理後再來檢查。
- expanded_urls，使用 Excel 查看內容發現有許多重複 url，目前不知道目的為何

_其他_
- Erroneous datatypes：timestamp 型別應改為 datetime
- source 和 Tweet JSON 的 source 相同，夾雜 html 標籤語言。

**Tidiness issues**
- doggo、floofer、pupper、puppo 內容是形容詞，可以改成 0, 1，此外還隱藏一個欄位，可以加總這 4 個欄位得知多個形容詞是否有不同結果
- rating_numerator, rating_denominator 還隱藏了一個欄位訊息 rating_numerator/rating_denominator 可以得到一個大致的比例

#### Image prediction table

**Quality issues**
-  有324筆數據判定不出是哪一種狗的品種

**Tidiness issues**
- 資料存在三種預測結果，裡面隱藏一個欄位即最終結果
- 這份資料可以跟 Twitter archive enhanced table 合併，將預測最好的結果（狗品種）合併過去。

## Clean

#### Tweet json table

- 移除不要的欄位，contributors、coordinates、geo、possibly_sensitive、possibly_sensitive_appealable、place、display_text_range、entities、favorited、retweeted、truncated、user
- 移除 extended_entities 缺失資料
- 刪除 in_reply_to_screen_name、in_reply_to_status_id、in_reply_to_status_id_str、in_reply_to_user_id、in_reply_to_user_id_str  有值的數據
- 刪除 is_quote_status 為 True 的數據
- 刪除 retweeted_status 有值的數據
- 檢查剩下資料，移除沒有資料的欄位
- 移除 is_quote_status、extended_entities 欄位
- 修正 id_str 型別為 str datatype
- 修正 lang 型別為 category datatype
- 修剪 source 內容不要 html 資訊

#### Twitter archive enhanced table
- 合併 Twitter archive enhanced table 和 Tweet json table。
- 合併後使用 info 檢查資訊，如果欄位沒有任何數據，可直接移除
- 計算 rating_numerator / rating_denominator 並放到新欄位 rating_ratio
- doggo、floofer、pupper、puppo 內容是形容詞，可以改成 0, 1，此外還隱藏一個欄位，可以加總這 4 個欄位得知多個形容詞是否有不同結果

### Image prediction table
- 刪除查無品種的數據（一共 324筆）
- 依照預測結果優先順序找出狗品種並放到新開欄位 prediction 
- 與 Twitter archive enhanced table 合併